In [1]:
import numpy as np
import ot
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# https://www.slideshare.net/joisino/ss-251328369
# p35

In [3]:
ws = np.array([0.2, 0.5, 0.2, 0.1])
wt = np.array([0.3, 0.3, 0.4, 0.0])

# https://stackoverflow.com/questions/58584413/black-formatter-ignore-specific-multi-line-code
# fmt: off
C = np.array([[0, 2, 2, 2],
              [2, 0, 1, 2],
              [2, 1, 0, 2],
              [2, 2, 2, 0]])
# fmt: on

In [4]:
# 汎用ソルバ・厳密ソルバによる解法
# 線形計画問題として定式化→シンプレックス法とかで解く
P_ = ot.emd(ws, wt, C)
cost = ot.emd2(ws, wt, C)  # (P * C).sum()
print(P_)
print(cost)

[[0.2 0.  0.  0. ]
 [0.  0.3 0.2 0. ]
 [0.  0.  0.2 0. ]
 [0.1 0.  0.  0. ]]
0.39999999999999997


In [5]:
"""
sinkhornでの解法
- 高速
- シンプル
- ただし、厳密な最適輸送の解は求まらない
"""
eps = 0.1  # 大きいと高速、小さいと厳密に
K = np.exp(-C / eps)  # Gibbs kernel
"""
u = exp(f/eps)
v = exp(g/eps)
"""
u = np.ones(4)
for i in range(100):
    v = wt / (K.T @ u)
    u = ws / (K @ v)
# f = eps * np.log(u + 1e-9)  # シンクホーン変数から元の双対変数へ
# g = eps * np.log(v + 1e-9)
"""
P = exp((f + g - C)/eps)
f, gが完璧に収束しない限り、Pは厳密には実行可能とは限らない
"""
P = u.reshape(4, 1) * K * v.reshape(1, 4)  # 主問題の解
np.set_printoptions(formatter={"float": "{:.2f}".format})
print(P)
print((P * C).sum())

[[0.20 0.00 0.00 0.00]
 [0.00 0.30 0.20 0.00]
 [0.00 0.00 0.20 0.00]
 [0.10 0.00 0.00 0.00]]
0.3993214951621453


In [17]:
"""
https://www.slideshare.net/joisino/ss-249394573
p218
シンクホーンは微分可能（特に、計算が行列演算のみのため、自動微分ライブラリが使える）
微分の使いどころの例：配置問題
倉庫: nコ
工場: mコ
j番目の工場の位置は(x_j, y_j)で、b_jグラムの小麦粉を要求
i番目の倉庫は小麦粉をa_iグラム保存可能
（倉庫から工場への総輸送コストを最小化するには、輸送行列をどのように設定し、）
各倉庫をどこに配置すればよいか？
→2つの問題を解く？

解き方
1.倉庫の位置変数をランダム初期化
2.C_i_j = 倉庫iと工場jの距離として計算
3.sinkhornに(a, b, C)を入力して総輸送コストを評価
4.倉庫の位置変数の勾配を求めて倉庫の位置を改善→2へ
"""

torch.manual_seed(0)

x = torch.rand(5, 2)
y = torch.rand(5, 2) + torch.FloatTensor([0, 2])
z = torch.rand(5, 2) + torch.FloatTensor([1, 1])
mu = torch.cat([x, y, z])
# step 1
# nu = nn.parameter.Parameter(torch.rand(12, 2) * 2)  # 位置変数のつもり
nu = torch.rand(12, 2) * 2
nu.requires_grad = True
n, m = len(mu), len(nu)
a = torch.ones(n) / n
b = torch.ones(m) / m

eps = 0.1
optimizer = optim.SGD([nu], lr=1.0)
for it in range(100):
    # step 2
    C = torch.linalg.norm(mu.reshape(n, 1, 2) - nu.reshape(1, m, 2), axis=2)
    K = torch.exp(-C / eps)
    u = torch.ones(n)
    for i in range(100):
        v = b / (K.T @ u)
        u = a / (K @ v)

    # f = eps * torch.log(u + 1e-9)
    # g = eps * torch.log(v + 1e-9)

    # step 3
    P = u.reshape(n, 1) * K * v.reshape(1, m)
    loss = (P * C).sum()
    # step 4
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

np.set_printoptions(formatter={"float": "{:.3f}".format})
print(P.detach().cpu().numpy())
print(loss)
# print(nu)

[[0.000 0.039 0.001 0.000 0.000 0.020 0.000 0.007 0.000 0.000 0.000 0.000]
 [0.000 0.000 0.067 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000]
 [0.000 0.006 0.009 0.000 0.000 0.003 0.000 0.048 0.000 0.000 0.000 0.000]
 [0.000 0.012 0.000 0.000 0.000 0.051 0.000 0.003 0.000 0.000 0.000 0.000]
 [0.000 0.026 0.006 0.000 0.000 0.009 0.000 0.025 0.000 0.000 0.000 0.000]
 [0.031 0.000 0.000 0.022 0.000 0.000 0.000 0.000 0.008 0.005 0.000 0.000]
 [0.001 0.000 0.000 0.007 0.000 0.000 0.000 0.000 0.000 0.059 0.000 0.000]
 [0.048 0.000 0.000 0.009 0.000 0.000 0.000 0.000 0.008 0.002 0.000 0.000]
 [0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.067 0.000 0.000 0.000]
 [0.004 0.000 0.000 0.045 0.000 0.000 0.000 0.000 0.000 0.017 0.000 0.000]
 [0.000 0.000 0.000 0.000 0.002 0.000 0.000 0.000 0.000 0.000 0.065 0.000]
 [0.000 0.000 0.000 0.000 0.058 0.000 0.000 0.000 0.000 0.000 0.008 0.000]
 [0.000 0.000 0.000 0.000 0.023 0.000 0.015 0.000 0.000 0.000 0.010 0.019]
 [0.000 0.000 0.000 0.000

In [18]:
a_ = a.detach().cpu().numpy()
b_ = b.detach().cpu().numpy()
C_ = C.detach().cpu().numpy()
P_ = ot.emd(a_, b_, C_)
cost = ot.emd2(a_, b_, C_)  # (P * C).sum()
print(P_)
print(cost)

[[0.000 0.050 0.000 0.000 0.000 0.017 0.000 0.000 0.000 0.000 0.000 0.000]
 [0.000 0.000 0.067 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.067 0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000 0.000 0.067 0.000 0.000 0.000 0.000 0.000 0.000]
 [0.000 0.033 0.017 0.000 0.000 0.000 0.000 0.017 0.000 0.000 0.000 0.000]
 [0.017 0.000 0.000 0.033 0.000 0.000 0.000 0.000 0.017 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.067 0.000 0.000]
 [0.067 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.067 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.050 0.000 0.000 0.000 0.000 0.000 0.017 0.000 0.000]
 [0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.067 0.000]
 [0.000 0.000 0.000 0.000 0.067 0.000 0.000 0.000 0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000 0.017 0.000 0.017 0.000 0.000 0.000 0.017 0.017]
 [0.000 0.000 0.000 0.000